In [ ]:
# To make sure CUDA is available and working
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

In [ ]:
#Download moondream2
from huggingface_hub import snapshot_download
import os

# Configuration
model_id = "vikhyatk/moondream2"
revision = "2025-06-21"  
local_folder = "./moondream_weights"

print(f"Starting download of {model_id} (Revision: {revision})...")

# Download the model files
local_model_path = snapshot_download(
    repo_id=model_id,
    revision=revision,
    local_dir=local_folder,
    local_dir_use_symlinks=False, 
    ignore_patterns=["*.md", "*.gitattributes"] 
)

print(f"✅ Download complete! Model saved to: {os.path.abspath(local_model_path)}")

In [ ]:
######################################## COPY FILES FROM CACHE ##############################################

import os
import shutil

# Source and destination paths
SOURCE_PATH = os.path.expanduser("~/moondream_weights")
BASE_DIR = Path(__file__).resolve().parent
SOURCE_PATH = BASE_DIR / "moondream_weights"

# Create the cache directory
os.makedirs(CACHE_PATH, exist_ok=True)

print(f"📁 Copying files from {SOURCE_PATH}")
print(f"📂 To cache location: {CACHE_PATH}")

# Copy all Python files and config files
files_to_copy = [
    "lora.py",
    "layers.py", 
    "configuration_moondream.py",
    "vision_encoder.py",
    "config.json",
    "region_model.py",
    "rope.py",
    "fourier_features.py"
]

for filename in files_to_copy:
    src = os.path.join(SOURCE_PATH, filename)
    dst = os.path.join(CACHE_PATH, filename)
    
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"✅ Copied {filename}")
    else:
        print(f"⚠️  {filename} not found (might not be needed)")

print("🎉 Setup complete! Try running your code again.")

In [1]:
################################################ LOAD MODEL ################################################################## 
import torch
import os
import sys


os.environ['TRANSFORMERS_OFFLINE'] = '1'
os.environ['HF_HUB_OFFLINE'] = '1'
os.environ['HF_DATASETS_OFFLINE'] = '1'

from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image, ImageDraw, ImageFont

print("⏳ Loading Model...")

MODEL_PATH = os.path.expanduser("~/moondream_weights")
sys.path.append(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, 
    trust_remote_code=True,
    local_files_only=True
).to("cuda").eval()

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH, 
    trust_remote_code=True,  
    local_files_only=True
)
print("✅ Model Loaded! Ready for images.")

⏳ Loading Model...


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [2]:
############################################# RESTITCH TILES ########################################################################
import re
import shutil

def restitch_and_draw(tile_dir, output_path, original_size, tile_size, overlap, detections):
    """
    tile_dir: Directory where tiles are stored
    output_path: Where to save the final stitched image
    original_size: Tuple (width, height) of the original large image
    tile_size: int (e.g., 512)
    overlap: int (e.g., 64)
    detections: List of dicts: {'tile_name': ..., 'boxes': [{'x_min', 'y_min', 'x_max', 'y_max'}, ...]}
    """
    # 1. Initialize a new canvas
    canvas = Image.new('RGB', original_size)
    step = tile_size - overlap
    
    # 2. Place tiles on the canvas (Restitching)
    for filename in sorted(os.listdir(tile_dir)):
        if not filename.endswith(('.tif', '.png')): continue
        
        # Parse row and column from filename (regex for _rXXXX_cXXXX)
        match = re.search(r'_r(\d+)_c(\d+)', filename)
        if match:
            row, col = int(match.group(1)), int(match.group(2))
            tile = Image.open(os.path.join(tile_dir, filename))
            
            # Global position calculation
            x_offset = col * step
            y_offset = row * step
            canvas.paste(tile, (x_offset, y_offset))

    # 3. Translate and Draw Bounding Boxes
    draw = ImageDraw.Draw(canvas)
    for det in detections:
        tile_name = det['tile_name']
        match = re.search(r'_r(\d+)_c(\d+)', tile_name)
        if not match: continue
        
        row, col = int(match.group(1)), int(match.group(2))
        x_offset = col * step
        y_offset = row * step
        
        for box in det['objects']:
            gx1 = (box['x_min'] * tile_size) + x_offset
            gy1 = (box['y_min'] * tile_size) + y_offset
            gx2 = (box['x_max'] * tile_size) + x_offset
            gy2 = (box['y_max'] * tile_size) + y_offset
            
            draw.rectangle([gx1, gy1, gx2, gy2], outline="red", width=5)

    shutil.rmtree(tile_dir)
    os.makedirs(tile_dir)
    canvas.save(output_path)
    print(f"Stitched image saved to {output_path}")

In [3]:
######################################## SPLITS IMAGES IN TILES ##############################################
import numpy as np
import argparse
from pathlib import Path

def tile_tiff_image(
    input_path,
    output_dir,
    tile_size=378,
    overlap=50,
    save_format='png',
    compression=None
):
    """
    Split a TIFF image into overlapping tiles.
    
    Parameters:
    -----------
    input_path : str
        Path to input TIFF image
    output_dir : str
        Directory to save tiles
    tile_size : int
        Size of each tile (width and height in pixels)
    overlap : int
        Number of pixels to overlap between adjacent tiles
    """
    
    os.makedirs(output_dir, exist_ok=True)
    
    img = Image.open(input_path)
    
    width, height = img.size
    print(f"Image size: {width} x {height}")
    print(f"Image mode: {img.mode}")
    print(f"Tile size: {tile_size} x {tile_size}")
    print(f"Overlap: {overlap} pixels")
    
    step = tile_size - overlap
    
    # Calculate number of tiles
    cols = (width - overlap + step - 1) // step
    rows = (height - overlap + step - 1) // step
    
    print(f"Creating {rows} x {cols} = {rows * cols} tiles")
    
    tile_count = 0
    
    for row in range(rows):
        for col in range(cols):
            # Calculate tile boundaries
            x_start = col * step
            y_start = row * step
            x_end = min(x_start + tile_size, width)
            y_end = min(y_start + tile_size, height)
            
            # Extract tile
            tile = img.crop((x_start, y_start, x_end, y_end))
            
            # Generate filename
            base_name = Path(input_path).stem
            tile_filename = f"{base_name}_tile_r{row:04d}_c{col:04d}.{save_format}"
            tile_path = os.path.join(output_dir, tile_filename)
            
            if save_format.lower() == 'tif':
                # Save as TIFF with optional compression
                save_params = {}
                if compression != None:
                    save_params['compression'] = compression.lower()
                tile.save(tile_path, **save_params)
            else:
                tile.save(tile_path)
            
            tile_count += 1
    
    print(f"\nComplete! Created {tile_count} tiles in '{output_dir}'")
    return width, height, tile_size, overlap

In [4]:
######################################## MAIN SCRIPT ##############################################
import io
import base64
import ollama

image_dir = "/home/kalina/moondream_finetuning/geoint_images/resized"
orig_width, orig_height, tile_size, overlap = tile_tiff_image("/mnt/c/Users/kalin/Downloads/gunter_data/R9C11_bccc1add-2c7a-3c95-8e0f-409376118522.tif",image_dir)
output_file_name = "result.png"

def outliers_detection(box, image_width, image_height, min_area_ratio=0.02, max_area_ratio=0.85):
    """
    Filter out detections that are too large (likely false positives selecting whole image)
    or too small (likely noise).
    """
    
    # Calculate detection dimensions
    det_width = (box['x_max'] - box['x_min']) * image_width
    det_height = (box['y_max'] - box['y_min']) * image_height
    det_area = det_width * det_height
    image_area = image_width * image_height
    area_ratio = det_area / image_area

    # Filter by area ratio
    if area_ratio < min_area_ratio or area_ratio > max_area_ratio:
        return True
    else:
        return False


def verify_with_ollama(image_path, target_object = "military vehicle/vehicles", ollama_host="http://10.255.255.254:11440"):
    """
    Sends an image to Ollama's Vision model for verification.
    """
    response = ollama.chat(
        model='llama3.2-vision:90b',
        messages=[{
            'role': 'user',
            'content': f"Do you see anything that looks like military vehicle or equipment in this image? Reply with 'Yes' or 'No' and a short reason.",
            'images': [image_path] 
        }]
    )
    content = response['message']['content']
    print(content.lower())
    is_verified = "yes" in content.lower()
    return is_verified, content
    
all_detections=[]

#Looping through tiles
for filename in os.listdir(image_dir):
    image = Image.open(os.path.join(image_dir,filename))
    width, height = image.size 
    target_object = "military vehicle/vehicles" 
    
    results = model.detect(image, target_object)
    objects = results["objects"]
    
    if objects:
        print(f"✅ Found {len(objects)} targets") 
        for box in objects:
                x1 = box['x_min']
                y1 = box['y_min']
                x2 = box['x_max']
                y2 = box['y_max']
                if outliers_detection(box, width, height):
                    print(f"❌ REJECTED: This tile is an outlier.")
                else:
                    is_verified, reason = verify_with_ollama(os.path.join(image_dir, filename))
                    if is_verified:
                        draw = ImageDraw.Draw(image)
                        all_detections.append({
                            'tile_name': filename,
                            'objects': [box]
                        })
                        draw.rectangle([x1*width, y1*height, x2*width, y2*height], outline="red", width=5)
                        image.show()
            
                    else: print(f"❌ REJECTED: {reason}")
    else:
        print("❌ No targets found.")
   
restitch_and_draw(image_dir,output_file_name,(orig_width, orig_height), tile_size, overlap, all_detections)

Image size: 4129 x 3037
Image mode: RGB
Tile size: 378 x 378
Overlap: 50 pixels
Creating 10 x 13 = 130 tiles

Complete! Created 130 tiles in '/home/kalina/moondream_finetuning/geoint_images/resized'
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
✅ Found 1 targets
❌ REJECTED: This tile is an outlier.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
❌ No targets found.
✅ Found 1 targets


ResponseError: model 'llama3.2-vision:90b' not found (status code: 404)

In [ ]:
######################################## DOWNLOADING IMAGES FROM THE BACKET ##############################################
import requests
import zipfile
import io
import os
from datetime import datetime

def fetch_and_download_geoint_data(since_date="2026-01-01T00:00:00Z", download_dir="geoint_images"):
    """
    Fetches GEOINT data from the API and downloads pictures from bucket URLs.
    
    Args:
        since_date: ISO format date string for the 'since' parameter
        download_dir: Directory where files will be downloaded
    """
    # Create download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)
    
    # API endpoint
    url = f"https://europe-west3-soldier-tracker.cloudfunctions.net/geoint-coms/read?since={since_date}"
    
    try:
        # Fetch data from API
        print(f"Fetching data from API...")
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        
        # Check if request was successful
        if data.get("status") != "success":
            print(f"API returned non-success status: {data.get('status')}")
            return
        
        print(f"Found {data.get('count', 0)} record(s)")
        
        # Process each record
        for record in data.get("data", []):
            uuid = record.get("uuid")
            bucket_url = record.get("bucket_url")
            insert_date = record.get("insert_date")
            
            print(f"\nProcessing record {uuid}")
            print(f"Insert date: {insert_date}")
            print(f"Downloading from: {bucket_url}")
            
            # Download the zip file
            zip_response = requests.get(bucket_url)
            zip_response.raise_for_status()
            
            # Create a directory for this specific record
            record_dir = os.path.join(download_dir, insert_date)
            os.makedirs(record_dir, exist_ok=True)
            
            # Extract zip file
            with zipfile.ZipFile(io.BytesIO(zip_response.content)) as zip_file:
                # Extract all files
                zip_file.extractall(record_dir)
                print(f"Extracted {len(zip_file.namelist())} file(s) to {record_dir}")
                
        print(f"\nDownload complete! Files saved to '{download_dir}' directory")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except zipfile.BadZipFile as e:
        print(f"Error extracting zip file: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    # Run the function
    fetch_and_download_geoint_data()